In [1]:
import numpy as np
from wrf import getvar, ALL_TIMES, interplevel, get_cartopy, to_np
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm

In [8]:
geo = Dataset('/media/lucasdmarten/HD/wrf/domains/firstDomainPC/geo_em.d01.nc')

In [32]:
wrfile = Dataset('/media/lucasdmarten/HD/wrf/WRF/run/wrfout_d01_2018-05-22_18:00:00')
wrfile

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    TITLE:  OUTPUT FROM WRF V4.2.2 MODEL
    START_DATE: 2018-05-22_18:00:00
    SIMULATION_START_DATE: 2018-05-22_18:00:00
    WEST-EAST_GRID_DIMENSION: 100
    SOUTH-NORTH_GRID_DIMENSION: 100
    BOTTOM-TOP_GRID_DIMENSION: 40
    DX: 15000.0
    DY: 15000.0
    AERCU_OPT: 0
    AERCU_FCT: 1.0
    IDEAL_CASE: 0
    DIFF_6TH_SLOPEOPT: 0
    AUTO_LEVELS_OPT: 2
    DIFF_6TH_THRESH: 0.1
    DZBOT: 50.0
    DZSTRETCH_S: 1.3
    DZSTRETCH_U: 1.1
    SKEBS_ON: 0
    SPEC_BDY_FINAL_MU: 1
    USE_Q_DIABATIC: 0
    GRIDTYPE: C
    DIFF_OPT: 1
    KM_OPT: 4
    DAMP_OPT: 0
    DAMPCOEF: 0.2
    KHDIF: 0.0
    KVDIF: 0.0
    MP_PHYSICS: 8
    RA_LW_PHYSICS: 4
    RA_SW_PHYSICS: 4
    SF_SFCLAY_PHYSICS: 7
    SF_SURFACE_PHYSICS: 2
    BL_PBL_PHYSICS: 7
    CU_PHYSICS: 1
    SF_LAKE_PHYSICS: 0
    SURFACE_INPUT_SOURCE: 1
    SST_UPDATE: 0
    GRID_FDDA: 0
    GFDDA_INTERVAL_M: 0
    GFDDA_END_H: 0
    GRID_SFDDA: 

In [33]:
for i in tqdm(list(range(1,10))):
    wrfile = Dataset('/media/lucasdmarten/HD/DADOS_TCC/RESULTADOS/CASO2/MEMBRO{}/wrfout_d01_2018-09-22_18:00:00'.format(i))
    wspd = getvar(wrfile,'wspd_wdir', timeidx=ALL_TIMES)[0]
    wdir = getvar(wrfile,'wspd_wdir', timeidx=ALL_TIMES)[1]
    z = getvar(wrfile, 'z', units='m')
    topo = getvar(geo,'HGT_M')
    topo = int(topo[49,49].values)
    speed_at67 = interplevel(wspd, z, 67+topo)[:,49,49]
    dir_at65 = interplevel(wdir, z, 65.5+topo)[:,49,49]
    
    speed_at88 = interplevel(wspd, z, 88+topo)[:,49,49]
    
    speed_at108 = interplevel(wspd, z, 108+topo)[:,49,49]
    dir_at106 = interplevel(wdir, z, 106.5+topo)[:,49,49]

    T2=list(getvar(wrfile,'T2', timeidx=ALL_TIMES)[:,49,49].values)
    PSFC=list(getvar(wrfile,'PSFC', timeidx=ALL_TIMES)[:,49,49].values/10**2)
    wspd10 = list(getvar(wrfile,'wspd_wdir10', timeidx=ALL_TIMES)[0,:,49,49].values)
    wspd67 = list(speed_at67.values)
    wspd88 = list(speed_at88.values)
    wspd108 = list(speed_at108.values)
    wdir10 = list(getvar(wrfile,'wspd_wdir10', timeidx=ALL_TIMES)[1,:,49,49].values)
    wdir65 = list(dir_at65.values)
    wdir106 = list(dir_at106.values)
    r2 = list(getvar(wrfile,'rh2', timeidx=ALL_TIMES)[:,49,49].values)
    td2 = list(getvar(wrfile,'td2', timeidx=ALL_TIMES)[:,49,49].values)
    times=list(pd.to_datetime(getvar(wrfile,'times', timeidx=ALL_TIMES)[:].values))

    pd.DataFrame(
        {"times":times,"T2":T2,"R2":r2,"TD2":td2,"PSFC":PSFC,
         "wspd10":wspd10,"wdir10":wdir10,
         "wspd67":wspd67,"wdir65":wdir65,
         "wspd88":wspd88,
         "wspd108":wspd108,"wdir106":wdir106}, dtype='int8'
    ).to_csv('./c2.m{}.csv'.format(i), index=False)

100%|██████████| 9/9 [13:21<00:00, 89.03s/it]
